### Look ahead ED with a flexibility for 4 time steps

In [220]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

### Defining the system
We will take 4 time steps, 3 gens, 1 flexibility and 4 different demands at each time step. Gen, and flex will have min and max value and have a cost.

In [221]:
ngens = 3 #Total gen
nperiods = 4 #Total Time
nflexs = 1 #Total flex
min_gen = [0,0,0] #Min Gen limit
max_gen = [50,100,150] #Max Gen Limit
cost = [40,15,10] #Cost of each Gen
demand = [250,300,320,300] #Demand at each time step
min_flex = [0] #Min flex limit
max_flex = [30] #Max flex Limit
cost_flex = [8] #Flex cost

#### Defining variable
Important to analyse the result.

In [222]:
generators = np.ones((ngens,), dtype=int)
flexibilities = np.ones((nflexs), dtype = int)
model = gp.Model('ED')
ngen = model.addVars(ngens, nperiods, vtype=GRB.BINARY, name="ngen")
output = model.addVars(ngens, nperiods, vtype=GRB.CONTINUOUS, name="genoutput") #Take Gen output
nflex = model.addVars(nflexs, nperiods, vtype=GRB.BINARY, name="nflex")
output_flex = model.addVars(nflexs, nperiods, vtype=GRB.CONTINUOUS, name="flexoutput") #Take Flex Output

#### Defining General Constraint
At first, we will define the constraint for gen, and flex limit.

In [223]:
#Counting total Gen
numgen = model.addConstrs(ngen[g, p] <= generators[g]
                         for g in range(ngens) for p in range(nperiods))

#Counting total flex
numflex = model.addConstrs(nflex[f, p] <= flexibilities[f]
                         for f in range(nflexs) for p in range(nperiods))

# Min Gen Limit
min_output = model.addConstrs((output[g, p] >= min_gen[g] * ngen[g, p])
                              for g in range(ngens) for p in range(nperiods))

#Max Gen Limit
max_output = model.addConstrs((output[g, p] <= max_gen[g] * ngen[g, p])
                              for g in range(ngens) for p in range(nperiods))

#Min Flex Limit
min_output_f = model.addConstrs((output_flex[f, p] >= min_flex[f] * nflex[f, p])
                              for f in range(nflexs) for p in range(nperiods))

#Max Flex Limit
max_output_f = model.addConstrs((output_flex[f, p] <= max_flex[f] * nflex[f, p])
                              for f in range(nflexs) for p in range(nperiods))

### Look-ahead flexibillity constraint (Important)
This constraint will look the future demand and allocate flexibility based on that. Note that, flexibility cost is given $8$, which is less than the generating units. So, if I don't look ahead, the system will always use the flexibility first. But due to this, it will not be like this. Let's see.

In [224]:
flex_cons = model.addConstrs(gp.quicksum(output_flex[f, p] for f in range(nflexs) for p in range(nperiods)) <= 30  for p in range(nperiods))

#### Meet demand constraint
This constraint will make sure that the sum of generating unit plus flexibility will meet the existing demand

In [225]:
meet_demand = model.addConstrs(gp.quicksum(output[g, p] for g in range(ngens))+gp.quicksum(output_flex[f, p] for f in range(nflexs)) == demand[p]
                               for p in range(nperiods))

#### Objective Model
The objective is to minimize the cost for generating units and the cost for utilizing the flexibility over the whole time steps.

In [226]:
g_f_cost = gp.quicksum(cost[g]*(output[g,p])
                    for g in range(ngens) for p in range(nperiods))+gp.quicksum(cost_flex[f]*(output_flex[f,p])
                    for f in range(nflexs) for p in range(nperiods))

In [227]:
model.setObjective(g_f_cost)
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 56 rows, 32 columns and 96 nonzeros
Model fingerprint: 0xe193d133
Variable types: 16 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [8e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 18160.000000
Presolve removed 56 rows and 32 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 17840 18160 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.784000000000e+04, best bound 1.784000000000e+04, gap 0.0000%


### Getting Results
We will analyze the output from gen and the usage of flexibility in each time step for meeting demand

In [228]:
v = model.getVars()

In [229]:
for t in range(nperiods):
    print('=== Time Period: %g === \n  MW Demand: %g' %(t, demand[t]))
    for gen in range(ngens):
        gen_output =v[12+t+4*gen].X
        print(' Gen %2g output: %.0f' %(gen+1, gen_output))

    for fl in range(nflexs):
        output_flex = v[28 + t + 4 * fl].X
        print(' Flex %2g output: %.0f' %(fl+1, output_flex))

=== Time Period: 0 === 
  MW Demand: 250
 Gen  1 output: 0
 Gen  2 output: 100
 Gen  3 output: 150
 Flex  1 output: 0
=== Time Period: 1 === 
  MW Demand: 300
 Gen  1 output: 50
 Gen  2 output: 100
 Gen  3 output: 150
 Flex  1 output: 0
=== Time Period: 2 === 
  MW Demand: 320
 Gen  1 output: 40
 Gen  2 output: 100
 Gen  3 output: 150
 Flex  1 output: 30
=== Time Period: 3 === 
  MW Demand: 300
 Gen  1 output: 50
 Gen  2 output: 100
 Gen  3 output: 150
 Flex  1 output: 0


### Evaluating the result
Note that the demand for Time Period $2$ is maximum. Gen 1 cost is higher than all. Flexibility cost is low but it has a look ahead constraint.

Time Period 0: Total Demand: 250, Met by Gen 2 and Gen 3. It could use the flexibility, but it didn't.


Time Period 1: Total Demand: 300, Met by Gen 1, 2 and Gen 3. It could use the flexibility, but it didn't.


Time Period 2: Total Demand: 320, Met by Gen 1, 2 and Gen 3 and Flexibility. Note that, if the system utilized the flexibility in earlier time period, in Time Period 2, it needs to produce 70 MW from Gen 1, which is not cost effective for the whole time periods. We can only use 30 MW of total flexibility over the whole time periods. So, it is using in such a way that can minimize the total cost.


Time Period 3: We don't have anymore flexibility left. Total demand met by the Gen.


$\textbf{Summary}$: If we remove the look-ahead flexibility constraint, we will see that the system will use the fleixibility in all time step.

